## Libraries

In [2]:
from time import time
import sys
import os

from six.moves import xrange
import tensorflow as tf


import math
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

DATA_FILE_PATH = '../data/statefarmchallenge'

## Convert to recoreds

In [ ]:
def convert(images, labels, name):
    '''
    Convert 
    '''
    num_examples = labels.shape[0]
    if images.shape[0] != num_examples:
        raise ValueError("images size {} did not match labels size {}"
                         .format(images.shape[0], num_examples))
        
        
    
    
def main():

## Input Pipeline 

In [1]:
def read_file(queue):
    # Reader to read csv rows
    reader = tf.TextLineReader()
    key, value = reader.read(queue)
    
    record_defaults = [[''], [''], ['']]
    subject, label, img_file = tf.decode_csv(value, record_defaults=record_defaults)
    
def process_images(image):
    img_file = tf.read_file(path)
    img_data = tf.image.decode_jpeg(img_file, channels=3)
    resized = tf.image.resize_images(img_data, 256, 256)
    # scaled_croped = tf.image.crop_to_bounding_box(scaled, 0, 43, 256, 256)

    # Convert from [0, 255] -> [-0.5, 0.5] floats.
    image = tf.cast(resized, tf.float32) * (1. / 255.) - 0.5
    
    return image


    
def input_pipeline():
    # Create queue for csv file
    file_queue = tf.train.string_input_producer([DATA_FILE_PATH], num_epochs=num_epochs)

    image, label = decode_and_process_img(file_queue)

    images, labels = tf.train.shuffle_batch([image, label], batch_size=batch_size, 
                                            num_threads=2,
                                            capacity=1000 + 3 * batch_size,
                                            min_after_dequeue=1000)

    return image_batch, label_batch




def decode_and_process_img(queue):
    # Reader to read csv rows
    reader = tf.TextLineReader()
    key, value = reader.read(queue)
    
    record_defaults = [[''], [''], ['']]
    subject, label, img_file = tf.decode_csv(value, record_defaults=record_defaults)
    
    path = tf.cast('/data/SFC/imgs/train/' + label + '/' + img_file, tf.string)
    img_file = tf.read_file(path)
    img_data = tf.image.decode_jpeg(img_file, channels=3)
    resized = tf.image.resize_images(img_data, 256, 256)
#    scaled_croped = tf.image.crop_to_bounding_box(scaled, 0, 43, 256, 256)

    # Convert from [0, 255] -> [-0.5, 0.5] floats.
    image = tf.cast(resized, tf.float32) * (1. / 255.) - 0.5

    return image, label

    
def inputs(batch_size, num_epochs):
    with tf.name_scope('input'):
        # Create queue for csv file
        file_queue = tf.train.string_input_producer([DATA_FILE_PATH], num_epochs=num_epochs)

        image, label = decode_and_process_img(file_queue)
        
        images, labels = tf.train.shuffle_batch([image, label], batch_size=batch_size, 
                                                num_threads=2,
                                                capacity=1000 + 3 * batch_size,
                                                min_after_dequeue=1000)
    
    return images, labels

    


IndentationError: expected an indented block (<ipython-input-1-62efe6f51668>, line 22)

Use tf records to run our model as a threaded queue

## Helper Functions

In [ ]:
'''
Auxilary functions
'''


def variabel_on_cpu():
    with tf.device('\cpu0:'):
        
    
    return tf.Variable()


def variable_with_weight_decay():
    return var

TODO: 
 - add norms to layers
 - add images to summary
 - change learning rate to exp step
 - normalize images
 - experiment with color/grayscale
 - examine learning 

## Model

In [2]:
NUM_CLASSES = 10
batch_size = 128
patch_size = 10
depth1 = 32
depth2 = 64
depth3 = 64
num_hidden = 2048
learning_rate = .01
num_channels = 1
image_size = 256 * 256
num_epochs = 2


def inference(images, keep_prob):
    """
    Args:
    images: Images placeholder, from inputs().
    Returns:
    softmax_linear: Output tensor with the computed logits.
    """

    # Conv1 /w pooling
    with tf.name_scope('conv1') as scope:
        weights = tf.Variable(tf.truncated_normal(
              [patch_size, patch_size, num_channels, depth1], stddev=0.01))
        biases = tf.Variable(tf.zeros([depth1]))
        conv = tf.nn.conv2d(images, weights, [1, 1, 1 ,1], padding='SAME', name='conv1')
        conv1 = tf.nn.relu(conv, name=scope)
        
    pool1 = tf.nn.avg_pool(conv1, [1, 6, 6, 1], [1, 2, 2, 1], padding='SAME', name='pool1')
    norm1 = tf.nn.local_response_normalization(pool1, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75, name='norm1')

    # Conv2 /w pooling 
    with tf.name_scope('conv2') as scope:
        weights = tf.Variable(tf.truncated_normal(
              [patch_size, patch_size, depth1, depth2], stddev=0.01))
        biases = tf.Variable(tf.constant(1.0, shape=[depth2]))
        conv = tf.nn.conv2d(norm1, weights, [1, 1, 1, 1], padding='SAME', name='conv2')
        conv2 = tf.nn.relu(conv, name=scope)
    
    pool2 = tf.nn.avg_pool(conv2, [1, 6, 6, 1], [1, 2, 2, 1], padding='SAME', name='pool2')
    norm2 = tf.nn.local_response_normalization(pool2, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75, name='norm2')
    
    # Conv3 /w pooling
    with tf.name_scope('conv3') as scope:
        weights = tf.Variable(tf.truncated_normal(
              [patch_size, patch_size, depth2, depth3], stddev=0.01))
        biases = tf.Variable(tf.constant(1.0, shape=[depth3]))
        conv = tf.nn.conv2d(norm2, weights, [1, 1, 1, 1], padding='SAME', name='conv3')
        conv3 = tf.nn.relu(conv, name=scope)
    
    pool3 = tf.nn.avg_pool(conv3, [1, 6, 6, 1], [1, 2, 2, 1], padding='SAME', name='pool3')
    norm3 = tf.nn.local_response_normalization(pool3, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75, name='norm3') 
    shape = norm3.get_shape().as_list()
    reshape = tf.reshape(norm3, [shape[0], shape[1] * shape[2] * shape[3]])
        
        
    #local4
    with tf.name_scope('local3') as scope:
        weights = tf.Variable(
            tf.truncated_normal([image_size, num_hidden],
                                stddev=1.0 / math.sqrt(float(num_hidden))))

        biases = tf.Variable(tf.zeros([num_hidden]))
        z = tf.matmul(reshape, weights) + biases
        local4 = tf.nn.relu(z, name=scope)
        drop = tf.nn.dropout(local4, keep_prob)

    # local5
    with tf.name_scope('local4') as scope:
        weights = tf.Variable(
            tf.truncated_normal([num_hidden, num_hidden],
                                stddev=1.0 / math.sqrt(float(num_hidden))), name='weights_5')

        biases = tf.Variable(tf.zeros([num_hidden]), name='biases_5')
        z = tf.matmul(drop, weights) + biases
        local5 = tf.nn.relu(z, name=scope)
        drop = tf.nn.dropout(local5, keep_prob)
        
        
    # Linear
    with tf.name_scope('softmax_linear'):
        weights = tf.Variable(
            tf.truncated_normal([num_hidden, NUM_CLASSES],
                                stddev=1.0 / math.sqrt(float(num_hidden))), name='weights_s')

        biases = tf.Variable(tf.zeros([NUM_CLASSES]), name='biases_s')
        logits = tf.matmul(drop, weights) + biases
        
    return logits


def loss(logits, labels):
    """"
    Calculates the loss from the logits and the labels.
    Args:
    logits: Logits tensor, float - [batch_size, NUM_CLASSES].
    labels: Labels tensor, int64 - [batch_size].
    Returns:
    loss: Loss tensor of type float.
    """
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits, labels, name='xentropy')
    loss = tf.reduce_mean(cross_entropy, name='xentropy_mean')
    return loss


def training(loss, learning_rate):
    """
    Sets up the training Ops.
    Creates a summarizer to track the loss over time in TensorBoard.
    Creates an optimizer and applies the gradients to all trainable variables.
    The Op returned by this function is what must be passed to the
    `sess.run()` call to cause the model to train.
    Args:
    loss: Loss tensor, from loss().
    learning_rate: The learning rate to use for gradient descent.
    Returns:
    train_op: The Op for training.
    """
    # Add a scalar summary for the snapshot loss. (Tensor board)
    tf.scalar_summary('loss', loss)

    # Create a variable to track the global step.
    global_step = tf.Variable(0, name='global_step', trainable=False)

    # decay on our learning rate as we progress with training
#     TODO: Use step for learn rate
    learn_rate = tf.train.exponential_decay(learning_rate, global_step, 1000, 0.8)
    tf.scalar_summary('learn rate', learning_rate)
    
    # Create the gradient Adam optimizer with the given learning rate.
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    
    # Use the optimizer to apply the gradients that minimize the loss
    # (and also increment the global step counter) as a single training step.
    train_op = optimizer.minimize(loss, global_step=global_step)

    return train_op


def evaluation(logits, labels):
    """Evaluate the quality of the logits at predicting the label.
    Args:
    logits: Logits tensor, float - [batch_size, NUM_CLASSES].
    labels: Labels tensor, int32 - [batch_size], with values in the
      range [0, NUM_CLASSES).
    Returns:
    A scalar int32 tensor with the number of examples (out of batch_size)
    that were predicted correctly.
    """
    # For a classifier model, we can use the in_top_k Op.
    # It returns a bool tensor with shape [batch_size] that is true for
    # the examples where the label is in the top k (here k=1)
    # of all logits for that example.
    correct = tf.nn.in_top_k(logits, labels, 1)
    # Return the number of true entries.
    return tf.reduce_sum(tf.cast(correct, tf.int32))


## Train

In [ ]:
def run_training():
    tf.reset_default_graph()
    with tf.Graph().as_default():
        
        keep_prob = tf.placeholder(tf.float32)
        
        images, labels = inputs(batch_size, num_epochs)
        
        logits = inference(images, keep_prob)
                
        loss_op = loss(logits, labels)
        
        train_op = training(loss_op, learning_rate)

        init_op = tf.initialize_all_variables()


        sess = tf.Session()
        sess.as_default()
        
        sess.run(init_op)

        # Start populating the filename queue.
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(sess=sess, coord=coord)
        
        for i in range(10):
            # Retrieve a single instance:
            _, ls = sess.run([train_op, loss_op], feed_dict={keep_prob : 0.5})


        coord.request_stop()
        coord.join(threads)
        sess.close()
        
        
run_training()

## Test 